In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b21c4f98ada412d909d55d347ea215a3dd07d82ba6592fdb8e76d19d3e841b25
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
item = spark.read.parquet('/content/drive/MyDrive/Python Spark/item_his.parquet', header=True)
member = spark.read.parquet('/content/drive/MyDrive/Python Spark/member.parquet', header=True)
point = spark.read.parquet('/content/drive/MyDrive/Python Spark/point_his.parquet', header=True)
study = spark.read.parquet('/content/drive/MyDrive/Python Spark/study_his.parquet', header=True)
reg = spark.read.parquet('/content/drive/MyDrive/Python Spark/regdate_his.parquet', header=True)

In [ ]:
# 1. 각 회원상태에 대해 codename들에 대한 count

member.printSchema()
item.printSchema()

root
 |-- idx: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- status: string (nullable = true)
 |-- grade: string (nullable = true)

root
 |-- idx: string (nullable = true)
 |-- lv: string (nullable = true)
 |-- proc_ym: string (nullable = true)
 |-- proc_ymd: string (nullable = true)
 |-- codename: string (nullable = true)
 |-- mascodename: string (nullable = true)
 |-- payprice: string (nullable = true)
 |-- price: string (nullable = true)



In [ ]:
item_member = item.join(member,['idx'])
item_member.show(5)

+-----+-----+-------+--------+----------+--------------+--------+-----+---+--------+-------+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|sex|  status|  grade|
+-----+-----+-------+--------+----------+--------------+--------+-----+---+--------+-------+
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50| 남|유료회원|  0학년|
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50| 남|유료회원|초1학년|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50| 남|유료회원|  0학년|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50| 남|유료회원|초1학년|
|20163|161.0| 202304|20230408|  액세서리|아바타파츠구분|      50|   50| 여|유료회원|초2학년|
+-----+-----+-------+--------+----------+--------------+--------+-----+---+--------+-------+
only showing top 5 rows



In [ ]:
item_group = item_member.groupby('status', 'codename').agg(F.count(F.col('codename')).alias('codecount'))
item_group.orderBy('status', 'codename').show(62)

+--------+----------+---------+
|  status|  codename|codecount|
+--------+----------+---------+
|    복회|      상의|        1|
|    복회|상태메시지|        6|
|    복회|      신발|        6|
|    복회|  액세서리|        5|
|    복회|      얼굴|        2|
|    복회|      하의|        4|
|    복회|      헤어|        4|
|    신규|      상의|       56|
|    신규|상태메시지|      179|
|    신규|      신발|      114|
|    신규|  액세서리|       82|
|    신규|      얼굴|       92|
|    신규|    코스튬|       38|
|    신규|      하의|       49|
|    신규|      헤어|      170|
|유료회원|      상의|    18306|
|유료회원|상태메시지|    24459|
|유료회원|      신발|    28210|
|유료회원|  액세서리|    18359|
|유료회원|      얼굴|    22982|
|유료회원|    코스튬|     6605|
|유료회원|      하의|    15001|
|유료회원|      헤어|    37424|
|이관유료|      상의|       11|
|이관유료|상태메시지|       10|
|이관유료|      신발|       11|
|이관유료|  액세서리|        8|
|이관유료|      얼굴|        9|
|이관유료|    코스튬|        8|
|이관유료|      하의|       10|
|이관유료|      헤어|       17|
|    이월|      상의|       18|
|    이월|상태메시지|       13|
|    이월|      신발|       33|
|    이월|  

In [ ]:
# 2. 날짜별로 point를 가장 많이 힉득한 level

# point 테이블과 / item lv

point.printSchema()
item.printSchema()




root
 |-- idx: long (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- point: long (nullable = true)

root
 |-- idx: long (nullable = true)
 |-- lv: double (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- codename: string (nullable = true)
 |-- mascodename: string (nullable = true)
 |-- payprice: long (nullable = true)
 |-- price: long (nullable = true)



In [ ]:
point_item = point.join(item, ['idx', 'proc_ymd'])

point_item.show(5)
#해당 유저가 그 날짜에 lv을 찾을 수 있으니까!

+-----+--------+-------+-----+----+-------+----------+------------+--------+-----+
|  idx|proc_ymd|proc_ym|point|  lv|proc_ym|  codename| mascodename|payprice|price|
+-----+--------+-------+-----+----+-------+----------+------------+--------+-----+
|88572|20230408| 202304| 5000|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 4500|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 3000|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 3500|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 2500|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
+-----+--------+-------+-----+----+-------+----------+------------+--------+-----+
only showing top 5 rows



In [ ]:
point_items = point_item.groupby('proc_ymd', 'lv').agg(F.sum(F.col('point')).alias('point_sum'))

In [ ]:
point_items.orderBy('proc_ymd', 'lv').show()

+--------+----+---------+
|proc_ymd|  lv|point_sum|
+--------+----+---------+
|20230401| 2.0|     2000|
|20230401| 4.0|     9000|
|20230401| 6.0|    28000|
|20230401| 8.0|     6500|
|20230401|10.0|    52500|
|20230401|12.0|    82500|
|20230401|14.0|   138000|
|20230401|20.0|    55000|
|20230401|22.0|    34000|
|20230401|26.0|    82500|
|20230402| 6.0|    15000|
|20230402|11.0|    28500|
|20230402|22.0|    27500|
|20230403| 2.0|     3000|
|20230403| 3.0|     3000|
|20230403| 4.0|     6000|
|20230403| 5.0|    20000|
|20230403| 8.0|    28000|
|20230403| 9.0|    44000|
|20230403|10.0|     8500|
+--------+----+---------+
only showing top 20 rows



In [ ]:
point_items.show()

+--------+----+---------+
|proc_ymd|  lv|point_sum|
+--------+----+---------+
|20230524| 5.0|    10000|
|20230408|15.0|    10000|
|20230509|16.0|   302500|
|20230407|12.0|    38000|
|20230413|28.0|     9500|
|20230405| 7.0|    31500|
|20230424|13.0|    67500|
|20230406|12.0|   105000|
|20230406|17.0|     9500|
|20230518| 5.0|     3500|
|20230605|13.0|    49000|
|20230608|14.0|    10000|
|20230601|34.0|    27000|
|20230423|14.0|    19000|
|20230515| 7.0|     7500|
|20230629|20.0|    38000|
|20230518| 7.0|    42500|
|20230407| 8.0|    93000|
|20230615| 9.0|    30000|
|20230626|11.0|    76500|
+--------+----+---------+
only showing top 20 rows



In [ ]:
window_var = W.Window.partitionBy('proc_ymd').orderBy(F.desc('point_sum'))

point_item_window = point_items.withColumn('point_rank', F.row_number().over(window_var))

In [ ]:
point_item_window.printSchema()

root
 |-- proc_ymd: long (nullable = true)
 |-- lv: double (nullable = true)
 |-- point_sum: long (nullable = true)
 |-- point_rank: integer (nullable = false)



In [ ]:
point_item_window.filter(F.col('point_rank') == 1).show()

+--------+----+---------+----------+
|proc_ymd|  lv|point_sum|point_rank|
+--------+----+---------+----------+
|20230401|14.0|   138000|         1|
|20230402|11.0|    28500|         1|
|20230403|13.0|    57500|         1|
|20230404|10.0|   148000|         1|
|20230405|15.0|   202500|         1|
|20230406|10.0|   193500|         1|
|20230407| 6.0|   302000|         1|
|20230408|13.0|   113500|         1|
|20230409|11.0|   240000|         1|
|20230410|13.0|    68000|         1|
|20230411| 9.0|   179000|         1|
|20230412|19.0|   109500|         1|
|20230413|11.0|   317000|         1|
|20230414|11.0|   205000|         1|
|20230415|12.0|   182000|         1|
|20230416|16.0|    49000|         1|
|20230417|11.0|   146500|         1|
|20230418|23.0|   137500|         1|
|20230419|11.0|   135000|         1|
|20230420|14.0|   687000|         1|
|20230421|18.0|   110000|         1|
|20230422|11.0|   117500|         1|
|20230423| 7.0|   118500|         1|
|20230424|11.0|   155000|         1|
|

In [ ]:
# 3. pointnm 별로 획득한 point의 종류를 언더바로 이어서 보내주세요
# pointnm -> study라는 테이블에 있고, point 자체는  point에 있음
study.printSchema()
point.printSchema()

root
 |-- idx: long (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- pointnm: string (nullable = true)

root
 |-- idx: long (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- point: long (nullable = true)



In [ ]:
study_point = study.join(point, ['idx', 'proc_ymd'])


study_point.show()

+-----+--------+-------+-----------------+-------+-----+
|  idx|proc_ymd|proc_ym|          pointnm|proc_ym|point|
+-----+--------+-------+-----------------+-------+-----+
|94954|20230611| 202306|      한글 스피치| 202306| 5000|
|94954|20230611| 202306|      한글 스피치| 202306| 4000|
|94954|20230611| 202306|      한글 스피치| 202306| 4500|
|89734|20230418| 202304|      한글 스피치| 202304| 5000|
|89734|20230418| 202304|      한글 스피치| 202304| 5000|
|90082|20230423| 202304|      한글 스피치| 202304| 1500|
|87085|20230410| 202304|          AI 영어| 202304| 3500|
|87085|20230410| 202304|          AI 영어| 202304| 3000|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 1000|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 5000|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 1500|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 3500|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 2000|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 4500|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 3000|
|93198|20230519| 202305|도전! AI 받아쓰기| 202305| 2500|

In [ ]:
study_point = study_point.groupby('pointnm').agg(F.collect_set(F.col('point')).alias('point_set'))


In [ ]:
data1 = [3000,3500,1000, 500,4500]
data = []
for i in data1:
  data.append(str(i))

print(data)

['3000', '3500', '1000', '500', '4500']


In [ ]:
data = [str(i) for i in data1]
print(data)


['3000', '3500', '1000', '500', '4500']


In [ ]:
def underbar(var):
  str_var = []
  str_var = [str(i) for i in data1]


  return_var = '_'.join(str_var)

  return return_var

In [ ]:
underbar_udf = F.udf(underbar)

In [ ]:
study_point.withColumn('point_udf', underbar_udf(F.col('point_set'))).show(truncate=False)

+-----------------+-----------------------------------------------------------+-----------------------+
|pointnm          |point_set                                                  |point_udf              |
+-----------------+-----------------------------------------------------------+-----------------------+
|일기 쓰기        |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|학교 체험        |[5000]                                                     |3000_3500_1000_500_4500|
|중학 특강        |[500, 1500]                                                |3000_3500_1000_500_4500|
|AI 영어          |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|도전! AI 받아쓰기|[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|학교 공부 맛보기 |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|중학 신입생      |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_50